In [3]:
import os
import pandas as pd

DATAPATH = "../data"
CONFIGPATH = "../config"

# Get units to standardise
Get all the units used in the assays to standardise them according to UCUM guides

In [9]:
allunits = []

file_list = os.listdir(os.path.join(DATAPATH, "pathogen_original"))
for filename in file_list:
    df = pd.read_csv(os.path.join(DATAPATH, "pathogen_original", filename), low_memory=False)
    units = list(set(df["standard_units"]))
    allunits.extend(units)
allunits = list(set(allunits))
df = pd.DataFrame({"units": allunits})
df.to_csv(os.path.join(CONFIGPATH, "units.csv"), index=False) #this file will be processed manually in the UCUM website

In [58]:
#use this to check for specific assays in the downloaded data
file_list = os.listdir(os.path.join(DATAPATH, "pathogen_original"))
for filename in file_list:
    df = pd.read_csv(os.path.join(DATAPATH, "pathogen_original", filename), low_memory=False)
    print(filename)
    print(df["assay_chembl_id"][df["standard_type"]=="Percent Effect"])

mtuberculosis.csv
35203     CHEMBL4649971
35204     CHEMBL4649961
35268     CHEMBL4649971
35365     CHEMBL4649971
35366     CHEMBL4649961
              ...      
474920    CHEMBL4649972
474921    CHEMBL4649972
474922    CHEMBL4649972
474923    CHEMBL4649972
474924    CHEMBL4649972
Name: assay_chembl_id, Length: 398254, dtype: object
enterobacter.csv
Series([], Name: assay_chembl_id, dtype: object)
saureus.csv
Series([], Name: assay_chembl_id, dtype: object)
paeruginosa.csv
Series([], Name: assay_chembl_id, dtype: object)
abaumannii.csv
Series([], Name: assay_chembl_id, dtype: object)
kpneumoniae.csv
Series([], Name: assay_chembl_id, dtype: object)
efaecium.csv
Series([], Name: assay_chembl_id, dtype: object)
pfalciparum.csv
12603     CHEMBL4649943
12604     CHEMBL4649943
12605     CHEMBL4649943
12609     CHEMBL4649943
12610     CHEMBL4649943
              ...      
499887    CHEMBL4649945
499890    CHEMBL4649945
499893    CHEMBL4649945
499896    CHEMBL4649945
499899    CHEMBL4649964
Na

### Create functions to automatically convert units

In [1]:
f = "lambda x, y: x*y/1000"
f = eval(f)
f(3,2)

In [18]:
s = 'standard_value/(molecular_weight/1000)/1000'

def parse_function(s):
    if 'standard_value' not in s:
        return None
    if "molecular_weight" in s:
        p = "lambda x,y: "
    else:
        p = "lambda x: "
    s = s.replace("molecular_weight", "y")
    s = s.replace("standard_value", "x")
    s = p + s
    return eval(s)

f = parse_function(s)
f(1,2)

## Evaluate processed files to select relevant assays

In [100]:
file_list = os.listdir(os.path.join(DATAPATH, "pathogen_processed"))
dfs = []
for filename in file_list:
    df = pd.read_csv(os.path.join(DATAPATH, "pathogen_processed", filename), low_memory=False)
    dfs += [df]
df = pd.concat(dfs, ignore_index=True)

In [101]:
df['final_units'] = df['final_units'].fillna('N/A')  
au = df.groupby(['standard_type', 'final_units']).size().reset_index(name='count')
au_ = au[au["count"]>250] #only assays with over 250 results will be considered
au_.to_csv(os.path.join(CONFIGPATH,"st_type_summary.csv"), index=False) 

In [108]:
au = pd.read_csv(os.path.join(CONFIGPATH, "st_type_summary_manual.csv")) #DO NOT OVERWRITE THIS FILE
au = au[au["use"]!=0]

In [110]:
# Distribution of standard_value grouped by standard_type and standard_units
df_perc = df.groupby(['standard_type', 'final_units'])['final_value']\
            .describe(percentiles=[0.05, 0.1, 0.2, 0.25, 0.50, 0.75, 0.8, 0.9, 0.95])\
            .drop(columns=['mean', 'std', 'min', 'max'])\
            .sort_values(['count'], ascending=False)\
            .reset_index()

# df_value_by_unit.head(30)
# Show in alphabetical order (order case insensitive)
df_perc = df_perc[df_perc["count"]>250]
df_perc.sort_values(by=['standard_type', 'final_units'],
                                      key=lambda col: col.str.lower())

,standard_type,final_units,count,5%,10%,20%,25%,50%,75%,80%,90%,95%
44,% Control,%,324.0,7.112000e+01,77.000000,83.000000,85.000000,94.900000,100.000000,100.000000,100.000000,100.000000
31,AbsAC1000_uM,umol,505.0,1.044200e+00,1.698200,2.606200,3.206000,5.244000,6.425000,6.846000,8.784400,11.188000
34,AbsAC35,umol,443.0,4.246000e-01,1.144000,2.384000,3.460000,9.900000,21.065000,24.324000,34.296000,38.829000
9,AC50,umol,5870.0,7.404900e-01,1.591800,3.204400,4.022250,12.135000,80.537500,118.300000,380.000000,380.000000
8,Activity,%,6103.0,-2.300000e+00,0.010200,5.000000,8.800000,48.610000,88.000000,92.600000,99.900000,100.000000
23,Activity,log10CFU,954.0,-7.180000e-01,0.300000,1.000000,1.232500,2.710000,4.910000,5.432000,6.920000,8.710500
26,Activity,log10CFU/ml,619.0,-1.754000e+00,-1.100000,0.100000,0.510000,2.500000,4.000000,4.552000,6.000000,7.512000
38,Activity,N/A,389.0,7.600000e-08,0.020000,0.240000,0.390000,2.000000,23.000000,28.000000,360.000000,48798.332000
21,Activity,umol,1073.0,7.696989e-02,0.542137,2.806640,4.252804,26.900000,197.840374,259.391672,755.929984,3429.841630
7,EC50,umol,6735.0,1.200000e-02,0.046480,0.217000,0.357000,3.100000,15.000000,20.000000,100.000000,130.797613


In [111]:
au["5%"] = df_perc["5%"]
au["10%"] = df_perc["10%"]
au["20%"] = df_perc["20%"]
au["25%"] = df_perc["25%"]
au["50%"] = df_perc["50%"]
au["75%"] = df_perc["75%"]
au["80%"] = df_perc["80%"]
au["90%"] = df_perc["90%"]
au["95%"] = df_perc["95%"]

In [113]:
#threshold values are set at 10 and 50% aprox of the percentile distribution, unless there is a lot of values skewed
thresh = {
    ("AC50", "umol"):[1, 10],
    ("Activity", "%"):[50,90],
    ("Activity", "log10CFU"):[4, 10],
    ("Activity", "log10CFU/ml"):[1.5, 10],
    ("EC50", "umol"):[1.5, 10],
    ("ED50", "ug.mg-1"):[2, 10],
    ("ED50", "ug.mg-1"):[2, 10],
    ("ED50", "umol"):[1, 4],
    ("GI", "%"):[90, 50],
    ("IC50", "umol"):[2.5,10],
    ("IC90", "umol"):[2.5,10],
    ("Inhibition", "%"):[50,90],
    ("IZ", "mm"):[2,5],
    ("Ki", "umol"):[2.5,10],
    ("MBC", "umol"):[1,2.5],
    ("MBC99.9", "umol"):[1,2.5],
    ("MBEC", "umol"):[1,2.5],
    ("MEC", "umol"):[1,5],
    ("MIC", "umol"):[1,5],
    ("MIC100", "umol"):[1,2.5],
    ("MIC50", "umol"):[1,2.5],
    ("MIC80", "umol"):[1,5],
    ("MIC90", "umol"):[1,5],
    ("MIC95", "umol"):[1,5],
    ("MIC99", "umol"):[1,5], 
    ("MIC=>80", "umol"):[1,5],
    ("MIC=>90", "umol"):[1,5],
    ("MIC>90", "umol"):[1,5],
    ("MIC>99", "umol"):[1,5],
    ("Percent Effect", "%"):[50,90],
    ("Potency", "umol"):[1,5],
}

In [114]:
for i,r in au.iterrows():
    key = (r['standard_type'], r['final_units'])
    
    if key in thresh:
        au.at[i, 'low_cut'] = thresh[key][0]
        au.at[i, 'high_cut'] = thresh[key][1]
    else:
        au.at[i, 'low_cut'] = None
        au.at[i, 'high_cut'] = None

In [116]:
au.drop(columns=["comments", "use", "cut-off"], inplace=True)
au.to_csv(os.path.join(CONFIGPATH, "cutoff_config.csv"), index=False)